In [91]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 30)

### **data import**

In [ ]:
import os

os.listdir()

In [93]:
veg =  pd.read_excel('Fin_EVI,NDVI_시군구별(1-12월).xlsx')
veg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254 entries, 0 to 253
Columns: 198 entries, Hcode_VEG to 법정동코드
dtypes: float64(193), int64(3), object(2)
memory usage: 393.0+ KB


### **merge with Air pollutants**

**PM2.5**

In [94]:
ap = pd.read_csv('PM2.5_xgb_15_20_sggb.csv', encoding='euc-kr')
ap.head(5)

,year,month,sigungu_code_b,sido,sigungu,pm25_xgb
0,2015,1,11110,서울,종로구,26.459351
1,2015,1,11140,서울,중구,24.058521
2,2015,1,11170,서울,용산구,26.320779
3,2015,1,11200,서울,성동구,23.684910
4,2015,1,11215,서울,광진구,28.000963


In [95]:
veg.loc[veg['법정동=행정동이름'].str.startswith('서울특별시 광진구')][['법정동=행정동이름','법정동코드','행정동코드']].head(1)
# >> PM2.5는 법정동코드 기반으로 정리되어 있음 확인 가능.

,법정동=행정동이름,법정동코드,행정동코드
4,서울특별시 광진구,11215,11050


In [96]:
ap.year.unique()

array([2015, 2016, 2017, 2018, 2019, 2020], dtype=int64)

데이터 형태 동일하게 수정

In [97]:
pm25 = pd.DataFrame()

for y in range(2015, 2021):
    pm25_list = pd.DataFrame()
    
    for m in range(1, 13):
        idx = [i for i in ap.index if ap.loc[i, 'year'] == y and ap.loc[i, 'month'] == m]
        pm = pd.DataFrame({'sigungu_code_b':ap.loc[idx]['sigungu_code_b'].reset_index(drop=True),
                           f'{y}_month{m}_pm25':ap.loc[idx]['pm25_xgb'].reset_index(drop=True)}).sort_values(by='sigungu_code_b').drop(columns=['sigungu_code_b'])
        pm25_list =  pd.concat([pm25_list, pm], axis=1)
      
    pm25 = pd.concat([pm25,pm25_list], axis=1)

In [98]:
pm25[['sigungu_code_b','sido','sigungu']] = ap.drop_duplicates(subset='sigungu_code_b').reset_index(drop=True).sort_values(by='sigungu_code_b')[['sigungu_code_b','sido','sigungu']]

pm25['name_PM25'] = pm25['sido'].str.strip() + ' ' + pm25['sigungu'].str.strip()
pm25[['sido','sigungu','name_PM25']].head(5)

,sido,sigungu,name_PM25
0,서울,종로구,서울 종로구
1,서울,중구,서울 중구
2,서울,용산구,서울 용산구
3,서울,성동구,서울 성동구
4,서울,광진구,서울 광진구


In [99]:
clist = ['sido','sigungu', 'name_PM25', 'sigungu_code_b'] + [c for c in pm25.columns if 'month' in c]

pm25 = pm25[clist].rename(columns={'sigungu_code_b':'Bcode_pm25'})

경기도 부천, 청주시(&서원구), 창원시 >> 시군구 단위 없고 위 레벨만 존재 확인

In [ ]:
pm25.loc[pm25['name_PM25'].str.startswith('충북')]

In [101]:
pm25.to_excel('pm25_reshape.xlsx', index=False, header=True, encoding='euc-kr')


**data from monitoring sites**

In [ ]:
mn = pd.read_csv('Airpollutants_annual mean.csv', encoding='euc-kr')
mn.head(5)

In [103]:
pd.set_option('display.max_rows', 30)
pd.DataFrame(mn.columns)

,0
0,sido
1,sigungu
2,region
3,sido_code
4,sigungu_code_b
...,...
130,PM2_5_2016
131,PM2_5_2017
132,PM2_5_2018
133,PM2_5_2019


In [104]:
veg.loc[veg['법정동=행정동이름'].str.startswith('서울특별시 관악구')][['법정동=행정동이름','행정동코드','법정동코드']].head(1)
# >> SO2, NO2, CO, O3, PM10은 법정동코드와 행정동코드 모두 정리되어 있음 확인 가능.

,법정동=행정동이름,행정동코드,법정동코드
20,서울특별시 관악구,11210,11620


In [105]:
mn['name_monitoring'] =  mn['sido'].str.strip() + ' ' + mn['sigungu'].str.strip()

# PM2.5는 개인노출 있어서 여기서 연도별 pm25 농도는 제거함
mn = mn.drop(columns = ['lat','long','chs_num','sido','sigungu','region',
                        'PM2_5_2015','PM2_5_2016','PM2_5_2017','PM2_5_2018','PM2_5_2019','PM2_5_2020'])

pm25 + monitoring data

In [106]:
mn = mn.rename(columns={'sigungu_code_h':'Hcode_monitoring',
                        'sigungu_code_b':'Bcode_monitoring'})
total_ap = pm25.merge(mn, left_on='Bcode_pm25', right_on='Bcode_monitoring', how='outer')
total_ap.isnull().sum()

sido                 4
sigungu              4
name_PM25            4
Bcode_pm25           4
2015_month1_pm25     4
                    ..
PM10_2017           68
PM10_2018           50
PM10_2019            9
PM10_2020            1
name_monitoring      1
Length: 200, dtype: int64

In [107]:
total_ap.loc[total_ap['name_PM25'].isnull()][['name_monitoring','Bcode_monitoring']]

,name_monitoring,Bcode_monitoring
226,인천 미추홀구,28177.0
227,경북 울릉군,47940.0
228,제주 서귀포시,50130.0
229,제주 제주시,50110.0


In [108]:
total_ap.loc[total_ap['name_monitoring'].isnull()][['name_PM25','Bcode_pm25']]

,name_PM25,Bcode_pm25
51,인천 남구,28170.0


**특정 지역명 수정필요**

In [109]:
# 인천광역시 남구 -> 인천광역시 미추홀구
# 경북 울릉군, 제주 서귀포시, 제주 제주시 -> PM2.5 자료에는 존재하지 않아서 NaN으로 놔둠

pm25['name_PM25'] = pm25['name_PM25'].replace({'인천 남구':'인천광역시 미추홀구'})
pm25['Bcode_pm25'] = pm25['Bcode_pm25'].replace({28170.0:28177.0})

total_ap = pm25.merge(mn, left_on='Bcode_pm25', right_on='Bcode_monitoring', how='outer')
total_ap.isnull().sum()

sido                 3
sigungu              3
name_PM25            3
Bcode_pm25           3
2015_month1_pm25     3
                    ..
PM10_2017           67
PM10_2018           49
PM10_2019            8
PM10_2020            0
name_monitoring      0
Length: 200, dtype: int64

In [110]:
total_ap.to_excel('Fin_total_air_pollution.xlsx', index=False, header=True, encoding='euc-kr')

### **Vegetation + Air pollution**

In [111]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 30)

In [112]:
veg =  pd.read_excel('Fin_EVI,NDVI_시군구별(1-12월).xlsx')
tap =  pd.read_excel('Fin_total_air_pollution.xlsx')

In [113]:
veg.columns

Index(['Hcode_VEG', '2015_month1_EVI', '2015_month2_EVI', '2015_month3_EVI',
       '2015_month4_EVI', '2015_month5_EVI', '2015_month6_EVI',
       '2015_month7_EVI', '2015_month8_EVI', '2015_month9_EVI',
       ...
       '2022_month7_NDVI', '2022_month8_NDVI', '2022_month9_NDVI',
       '2022_month10_NDVI', '2022_month11_NDVI', '2022_month12_NDVI',
       'newNAME_VEG', '행정동코드', '법정동=행정동이름', '법정동코드'],
      dtype='object', length=198)

In [114]:
tap.columns

Index(['sido', 'sigungu', 'name_PM25', 'Bcode_pm25', '2015_month1_pm25',
       '2015_month2_pm25', '2015_month3_pm25', '2015_month4_pm25',
       '2015_month5_pm25', '2015_month6_pm25',
       ...
       'PM10_2012', 'PM10_2013', 'PM10_2014', 'PM10_2015', 'PM10_2016',
       'PM10_2017', 'PM10_2018', 'PM10_2019', 'PM10_2020', 'name_monitoring'],
      dtype='object', length=200)

In [115]:
exposure =  veg.merge(tap, left_on='행정동코드',right_on='Hcode_monitoring', how='outer')
exposure.isnull().sum()

Hcode_VEG           12
2015_month1_EVI     16
2015_month2_EVI     15
2015_month3_EVI     19
2015_month4_EVI     15
                  ... 
PM10_2017          101
PM10_2018           83
PM10_2019           42
PM10_2020           34
name_monitoring     34
Length: 398, dtype: int64

In [116]:
exposure.loc[exposure.행정동코드.isnull()][['행정동코드','Hcode_monitoring','name_monitoring']]

,행정동코드,Hcode_monitoring,name_monitoring
254,NaN,23030.0,인천 미추홀구
255,NaN,31010.0,경기 수원시
256,NaN,31020.0,경기 성남시
257,NaN,31040.0,경기 안양시
258,NaN,31090.0,경기 안산시
259,NaN,31100.0,경기 고양시
260,NaN,31190.0,경기 용인시
261,NaN,31320.0,경기 여주시
262,NaN,34010.0,충남 천안시
263,NaN,35010.0,전북 전주시


In [117]:
tap[tap['name_monitoring'].str.startswith('경남 통합창원시')][['Bcode_monitoring','Hcode_monitoring','name_monitoring']]

,Bcode_monitoring,Hcode_monitoring,name_monitoring
208,48120,38110,경남 통합창원시


In [118]:
veg[veg['법정동=행정동이름'].str.startswith('경기도 성남시')][['행정동코드','법정동코드','법정동=행정동이름']]

,행정동코드,법정동코드,법정동=행정동이름
78,31021,41131,경기도 성남시 수정구
79,31022,41133,경기도 성남시 중원구
80,31023,41135,경기도 성남시 분당구


경기도 수원시 생성

In [ ]:
suwon = veg.loc[veg['법정동=행정동이름'].str.startswith('경기도 수원시')].drop(columns=['행정동코드','법정동코드','Hcode_VEG','SIGUNGU_CD_NDVI','newNAME_VEG','법정동=행정동이름'])
suwon_veg = pd.DataFrame(suwon.values.mean(axis=0)).transpose()
suwon_clist = suwon_veg.columns.to_list()

add_list = ['행정동코드', '법정동코드','법정동=행정동이름'] # same order as in veg

suwon_veg['행정동코드'] = 31010
suwon_veg['법정동코드'] = 41110
suwon_veg['법정동=행정동이름'] = '경기도 수원시' 

suwon_veg = suwon_veg[add_list+suwon_clist]

veg = pd.concat([veg, suwon_veg], axis=0).sort_values(by='법정동=행정동이름')

veg[veg['법정동=행정동이름'].str.startswith('경기도 수원시')][['행정동코드','법정동코드','법정동=행정동이름','2015_month1_EVI','2015_month1_NDVI']] # 추가완료

경기도 성남시

In [ ]:
sungnam = veg.loc[veg['법정동=행정동이름'].str.startswith('경기도 성남시')].drop(columns=['행정동코드','법정동코드','Hcode_VEG','SIGUNGU_CD_NDVI','newNAME_VEG','법정동=행정동이름'])
sungnam_veg = pd.DataFrame(sungnam.mean(axis=0)).transpose()
sungnam_clist = sungnam_veg.columns.to_list()

add_list = ['행정동코드', '법정동코드','법정동=행정동이름'] # same order as in veg

sungnam_veg['행정동코드'] = 31020
sungnam_veg['법정동코드'] = 41130
sungnam_veg['법정동=행정동이름'] = '경기도 성남시' 

sungnam_veg = sungnam_veg[add_list+sungnam_clist]

veg = pd.concat([veg, sungnam_veg], axis=0).sort_values(by='법정동=행정동이름')

veg[veg['법정동=행정동이름'].str.startswith('경기도 성남시')][['행정동코드','법정동코드','법정동=행정동이름','2015_month1_EVI','2015_month1_NDVI']] # 추가완료

경기도 안양시

In [ ]:
anyang = veg.loc[veg['법정동=행정동이름'].str.startswith('경기도 안양시')].drop(columns=['행정동코드','법정동코드','Hcode_VEG','SIGUNGU_CD_NDVI','newNAME_VEG','법정동=행정동이름'])
anyang_veg = pd.DataFrame(anyang.mean(axis=0)).transpose()
clist = anyang_veg.columns.to_list()

add_list = ['행정동코드', '법정동코드','법정동=행정동이름'] # same order as in veg

anyang_veg['행정동코드'] = 31040
anyang_veg['법정동코드'] = 41170
anyang_veg['법정동=행정동이름'] = '경기도 안양시' 

anyang_veg = anyang_veg[add_list+clist]

veg = pd.concat([veg, anyang_veg], axis=0).sort_values(by='법정동=행정동이름')

veg[veg['법정동=행정동이름'].str.startswith('경기도 안양시')][['행정동코드','법정동코드','법정동=행정동이름','2015_month1_EVI','2015_month1_NDVI']] # 추가완료

경기도 안산시

In [ ]:
new = veg.loc[veg['법정동=행정동이름'].str.startswith('경기도 안산시')].drop(columns=['행정동코드','법정동코드','Hcode_VEG','SIGUNGU_CD_NDVI','newNAME_VEG','법정동=행정동이름'])
add_veg = pd.DataFrame(new.mean(axis=0)).transpose()
clist = add_veg.columns.to_list()

add_list = ['행정동코드', '법정동코드','법정동=행정동이름'] # same order as in veg

add_veg['행정동코드'] = 31090
add_veg['법정동코드'] = 41270
add_veg['법정동=행정동이름'] = '경기도 안산시' 

add_veg = add_veg[add_list+clist]

veg = pd.concat([veg, add_veg], axis=0).sort_values(by='법정동=행정동이름')

veg[veg['법정동=행정동이름'].str.startswith('경기도 안산시')][['행정동코드','법정동코드','법정동=행정동이름','2015_month1_EVI','2015_month1_NDVI']] # 추가완료

경기도 고양시

In [ ]:
new = veg.loc[veg['법정동=행정동이름'].str.startswith('경기도 고양시')].drop(columns=['행정동코드','법정동코드','Hcode_VEG','SIGUNGU_CD_NDVI','newNAME_VEG','법정동=행정동이름'])
add_veg = pd.DataFrame(new.mean(axis=0)).transpose()
clist = add_veg.columns.to_list()

add_list = ['행정동코드', '법정동코드','법정동=행정동이름'] # same order as in veg

add_veg['행정동코드'] = 31100
add_veg['법정동코드'] = 41280
add_veg['법정동=행정동이름'] = '경기도 고양시' 

add_veg = add_veg[add_list+clist]

veg = pd.concat([veg, add_veg], axis=0).sort_values(by='법정동=행정동이름')

veg[veg['법정동=행정동이름'].str.startswith('경기도 고양시')][['행정동코드','법정동코드','법정동=행정동이름','2015_month1_EVI','2015_month1_NDVI']] # 추가완료

경기도 용인시

In [ ]:
new = veg.loc[veg['법정동=행정동이름'].str.startswith('경기도 용인시')].drop(columns=['행정동코드','법정동코드','Hcode_VEG','SIGUNGU_CD_NDVI','newNAME_VEG','법정동=행정동이름'])
add_veg = pd.DataFrame(new.mean(axis=0)).transpose()
clist = add_veg.columns.to_list()

add_list = ['행정동코드', '법정동코드','법정동=행정동이름'] # same order as in veg

add_veg['행정동코드'] = 31190
add_veg['법정동코드'] = 41460
add_veg['법정동=행정동이름'] = '경기도 용인시' 

add_veg = add_veg[add_list+clist]

veg = pd.concat([veg, add_veg], axis=0).sort_values(by='법정동=행정동이름')

veg[veg['법정동=행정동이름'].str.startswith('경기도 용인시')][['행정동코드','법정동코드','법정동=행정동이름','2015_month1_EVI','2015_month1_NDVI']] # 추가완료

충청북도 청주시

In [ ]:
new = veg.loc[veg['법정동=행정동이름'].str.startswith('충청북도 청주시')].drop(columns=['행정동코드','법정동코드','Hcode_VEG','SIGUNGU_CD_NDVI','newNAME_VEG','법정동=행정동이름'])
add_veg = pd.DataFrame(new.mean(axis=0)).transpose()
clist = add_veg.columns.to_list()

add_list = ['행정동코드', '법정동코드','법정동=행정동이름'] # same order as in veg

add_veg['행정동코드'] = 33040
add_veg['법정동코드'] = 43110
add_veg['법정동=행정동이름'] = '충청북도 청주시' 

add_veg = add_veg[add_list+clist]

veg = pd.concat([veg, add_veg], axis=0).sort_values(by='법정동=행정동이름')

veg[veg['법정동=행정동이름'].str.startswith('충청북도 청주시')][['행정동코드','법정동코드','법정동=행정동이름','2015_month1_EVI','2015_month1_NDVI']] # 추가완료

충청남도 천안시

In [ ]:
new = veg.loc[veg['법정동=행정동이름'].str.startswith('충청남도 천안시')].drop(columns=['행정동코드','법정동코드','Hcode_VEG','SIGUNGU_CD_NDVI','newNAME_VEG','법정동=행정동이름'])
add_veg = pd.DataFrame(new.mean(axis=0)).transpose()
clist = add_veg.columns.to_list()

add_list = ['행정동코드', '법정동코드','법정동=행정동이름'] # same order as in veg

add_veg['행정동코드'] = 34010
add_veg['법정동코드'] = 44130
add_veg['법정동=행정동이름'] = '충청남도 천안시' 

add_veg = add_veg[add_list+clist]

veg = pd.concat([veg, add_veg], axis=0).sort_values(by='법정동=행정동이름')

veg[veg['법정동=행정동이름'].str.startswith('충청남도 천안시')][['행정동코드','법정동코드','법정동=행정동이름','2015_month1_EVI','2015_month1_NDVI']] # 추가완료

전라북도 전주시

In [ ]:
new = veg.loc[veg['법정동=행정동이름'].str.startswith('전라북도 전주시')].drop(columns=['행정동코드','법정동코드','Hcode_VEG','SIGUNGU_CD_NDVI','newNAME_VEG','법정동=행정동이름'])
add_veg = pd.DataFrame(new.mean(axis=0)).transpose()
clist = add_veg.columns.to_list()

add_list = ['행정동코드', '법정동코드','법정동=행정동이름'] # same order as in veg

add_veg['행정동코드'] = 35010
add_veg['법정동코드'] = 45110
add_veg['법정동=행정동이름'] = '전라북도 전주시' 

add_veg = add_veg[add_list+clist]

veg = pd.concat([veg, add_veg], axis=0).sort_values(by='법정동=행정동이름')

veg[veg['법정동=행정동이름'].str.startswith('전라북도 전주시')][['행정동코드','법정동코드','법정동=행정동이름','2015_month1_EVI','2015_month1_NDVI']] # 추가완료

경상북도 포항시

In [ ]:
new = veg.loc[veg['법정동=행정동이름'].str.startswith('경상북도 포항시')].drop(columns=['행정동코드','법정동코드','Hcode_VEG','SIGUNGU_CD_NDVI','newNAME_VEG','법정동=행정동이름'])
add_veg = pd.DataFrame(new.mean(axis=0)).transpose()
clist = add_veg.columns.to_list()

add_list = ['행정동코드', '법정동코드','법정동=행정동이름'] # same order as in veg

add_veg['행정동코드'] = 37010
add_veg['법정동코드'] = 47110
add_veg['법정동=행정동이름'] = '경상북도 포항시' 

add_veg = add_veg[add_list+clist]

veg = pd.concat([veg, add_veg], axis=0).sort_values(by='법정동=행정동이름')

veg[veg['법정동=행정동이름'].str.startswith('경상북도 포항시')][['행정동코드','법정동코드','법정동=행정동이름','2015_month1_EVI','2015_month1_NDVI']] # 추가완료

경상남도 창원시

In [ ]:
new = veg.loc[veg['법정동=행정동이름'].str.startswith('경상남도 창원시')].drop(columns=['행정동코드','법정동코드','Hcode_VEG','SIGUNGU_CD_NDVI','newNAME_VEG','법정동=행정동이름'])
add_veg = pd.DataFrame(new.mean(axis=0)).transpose()
clist = add_veg.columns.to_list()

add_list = ['행정동코드', '법정동코드','법정동=행정동이름'] # same order as in veg

add_veg['행정동코드'] = 38110
add_veg['법정동코드'] = 48120
add_veg['법정동=행정동이름'] = '경상남도 창원시' 

add_veg = add_veg[add_list+clist]

veg = pd.concat([veg, add_veg], axis=0).sort_values(by='법정동=행정동이름')

veg[veg['법정동=행정동이름'].str.startswith('경상남도 창원시')][['행정동코드','법정동코드','법정동=행정동이름','2015_month1_EVI','2015_month1_NDVI']] # 추가완료

행정코드/이름 변경 반영: 인천 미추홀구, 경기도 여주시 

In [130]:
tap['name_monitoring'] = tap['name_monitoring'].replace({'충북 통합청주시':'충북 청주시',
                                                         '경남 통합창원시':'경남 창원시'}) # 경기도 여주시 행정동코드 변경(2012년에 31280 으로 변경됨, 최신 구역코드로 지정)38110

tap['Hcode_monitoring'] = tap['Hcode_monitoring'].replace({31320:31280}) # 경기도 여주시 행정동코드 변경(2012년에 31280 으로 변경됨, 최신 구역코드로 지정)

tap['Hcode_monitoring'] = tap['Hcode_monitoring'].replace({23030:23090}) # 인천 미추홀구 행정동코드 변경(2018년에 23010 으로 변경됨, 최신 구역코드로 지정)
veg['행정동코드'] = veg['행정동코드'].replace({23030.0:23090}) # 인천 미추홀구 행정동코드 변경(2018년에 23010 으로 변경됨, 최신 구역코드로 지정)

**수정 후 다시 merge 시도**

In [131]:
tap[tap['name_monitoring'].str.startswith('인천 미추홀구')][['Bcode_monitoring','Hcode_monitoring','name_monitoring']]

,Bcode_monitoring,Hcode_monitoring,name_monitoring
51,28177,23090,인천 미추홀구


In [132]:
veg[veg['법정동=행정동이름'].str.startswith('인천광역시 미추홀구')][['행정동코드','법정동코드','법정동=행정동이름']]

,행정동코드,법정동코드,법정동=행정동이름
51,23090,28177,인천광역시 미추홀구


In [133]:
exposure_new =  veg.merge(tap, left_on='행정동코드',right_on='Hcode_monitoring', how='outer')
exposure_new =  exposure_new.drop_duplicates(subset='행정동코드') # 청주시가 2개 였음

pd.set_option('display.max_rows', 10)
exposure_new.isnull().sum()

Hcode_VEG          10
2015_month1_EVI     5
2015_month2_EVI     4
2015_month3_EVI     8
2015_month4_EVI     4
                   ..
PM10_2017          99
PM10_2018          81
PM10_2019          40
PM10_2020          32
name_monitoring    32
Length: 590, dtype: int64

In [ ]:
pd.set_option('display.max_rows', None)
exposure_new.loc[exposure_new.name_monitoring.isnull()][['행정동코드','법정동=행정동이름','Hcode_monitoring','name_monitoring','2015_month1_pm25','2015_month8_pm25','PM10_2015','NO2_2015','SO2_2015']]

**Air pollution Imputation**

경기도 고양시, 경기도 성남시, 경기도 수원시, 경기도 안산시, 경기도 안양시, 경기도 용인시, 경상남도 창원시, 경상남도 포항시, 전라북도 전주시, 충청남도 천안시, 충청북도 청주시

In [135]:
missmatch_list = ['경기도 고양시','경기도 성남시', '경기도 수원시', '경기도 안산시', '경기도 안양시', '경기도 용인시',
                  '경상남도 창원시', '경상남도 포항시', '전라북도 전주시', '충청남도 천안시', '충청북도 청주시']

for miss in missmatch_list:
    print(f'impuation in {miss}')
    data = exposure_new.loc[exposure_new['법정동=행정동이름'] == miss, '2015_month1_pm25':'PM10_2020'].values

    exposure_new.loc[exposure_new['법정동=행정동이름'].str.startswith(miss), '2015_month1_pm25':'PM10_2020'] = data

# Double check
exposure_new.loc[exposure_new['법정동=행정동이름'].str.startswith('전라북도 전주시'), '2015_month1_pm25':'PM10_2020']

impuation in 경기도 고양시
impuation in 경기도 성남시
impuation in 경기도 수원시
impuation in 경기도 안산시
impuation in 경기도 안양시
impuation in 경기도 용인시
impuation in 경상남도 창원시
impuation in 경상남도 포항시
impuation in 전라북도 전주시
impuation in 충청남도 천안시
impuation in 충청북도 청주시


,2015_month1_pm25,2015_month2_pm25,2015_month3_pm25,2015_month4_pm25,2015_month5_pm25,2015_month6_pm25,2015_month7_pm25,2015_month8_pm25,2015_month9_pm25,2015_month10_pm25,...,PM10_2011,PM10_2012,PM10_2013,PM10_2014,PM10_2015,PM10_2016,PM10_2017,PM10_2018,PM10_2019,PM10_2020
225,32.265946,36.354287,35.306902,25.131476,26.313022,28.071941,20.736681,21.384742,16.849905,21.191836,...,49.150596,48.612792,57.4074,53.217886,49.479252,54.24335,51.807865,46.360318,46.315838,34.010916
226,32.265946,36.354287,35.306902,25.131476,26.313022,28.071941,20.736681,21.384742,16.849905,21.191836,...,49.150596,48.612792,57.4074,53.217886,49.479252,54.24335,51.807865,46.360318,46.315838,34.010916
227,32.265946,36.354287,35.306902,25.131476,26.313022,28.071941,20.736681,21.384742,16.849905,21.191836,...,49.150596,48.612792,57.4074,53.217886,49.479252,54.24335,51.807865,46.360318,46.315838,34.010916


In [136]:
exposure_new.to_excel('Fin_Imputed_vegetation_airpollution.xlsx', index=False, header=True, encoding='euc-kr')